In [1]:
import pandas as pd
import csv , cv2
import numpy as np
import tensorflow, keras
from keras.layers import Input, Conv2D, Activation, MaxPool2D,  Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
import matplotlib
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\XTREME\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\XTREME\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\XTREME\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\XTREME\.conda\envs\base2\lib\site-packages\tensorflow\python\framework

In [6]:
dru_train = pd.read_csv(r"C:\Users\XTREME\Desktop\selected\Selected_Projects\Drugreview\drugsComTrain_raw.tsv", sep = "\t")
dru_test = pd.read_csv(r"C:\Users\XTREME\Desktop\selected\Selected_Projects\Drugreview\drugsComTest_raw.tsv", sep = "\t")
#type(dru_train)len(dru_train)
print('Training length: ',dru_train.shape)
len(dru_test)
print('Testing length:',dru_test.shape)

Training length:  (161297, 7)
Testing length: (53766, 7)


In [7]:
dru = pd.concat([dru_train,dru_test])
dru.shape

(215063, 7)

In [8]:
dru.head()

,ID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [10]:
# dru['date'] = pd.to_datetime(dru['date'])
# dru['date'].head()
dru_train = dru[['ID', 'review','rating']].copy()
dru_train.head()

,ID,review,rating
0,206461,"""It has no side effect, I take it in combinati...",9.0
1,95260,"""My son is halfway through his fourth week of ...",8.0
2,92703,"""I used to take another oral contraceptive, wh...",5.0
3,138000,"""This is my first time using any form of birth...",8.0
4,35696,"""Suboxone has completely turned my life around...",9.0


In [12]:
dru_train.info(null_counts= True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215063 entries, 0 to 53765
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      215063 non-null  int64  
 1   review  215063 non-null  object 
 2   rating  215063 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.6+ MB


C:\Users\XTREME\.conda\envs\base2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  """Entry point for launching an IPython kernel.


In [17]:
print('ID unique values:',dru_train['ID'].unique())
print('ID Count:',dru_train['ID'].count())
print('ID Non-unique values: ',dru_train['ID'].nunique())

ID unique values: [206461  95260  92703 ... 130945  47656 113712]
ID Count: 215063
ID Non-unique values:  215063


In [18]:
!pip install vaderSentiment

In [17]:
import nltk
nltk.download(['punkt','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\XTREME\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\XTREME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [25]:
dru_train.head()

,ID,review,rating,cleanReview,ReviewScore
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403


In [26]:
 # remove stopwords from review
dru_train['cleanReview'] = dru_train['review'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))    
dru_train.head()

,ID,review,rating,cleanReview,ReviewScore
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403


In [22]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [27]:
dru_train['Review Score'] = dru_train['cleanReview'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
dru_train.head()

,ID,review,rating,cleanReview,ReviewScore,Review Score
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000,0.0000
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070,0.9070
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096,0.7096
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184,0.7184
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403,0.9403


In [28]:
positive_num = len(dru_train[dru_train['Review Score'] >=0.05])
neutral_num = len(dru_train[(dru_train['Review Score'] >-0.05) & (dru_train['Review Score']<0.05)])
negative_num = len(dru_train[dru_train['Review Score']<=-0.05])

positive_num,neutral_num,negative_num

(106198, 9035, 99830)

In [29]:
dru_train['vader Sentiment']= dru_train['Review Score'].map(lambda x:int(2) if x>=0.05 else int(1) if x<=-0.05 else int(0))
dru_train['vader Sentiment'].value_counts()

Total_vaderSentiment = positive_num + neutral_num + negative_num
Total_vaderSentiment

215063

In [30]:
dru_train.loc[dru_train['Review Score'] >=0.05,"vader Sentiment Label"] ="positive"
dru_train.loc[(dru_train['Review Score'] >-0.05) & (dru_train['Review Score']<0.05),"vader Sentiment Label"]= "neutral"
dru_train.loc[dru_train['Review Score']<=-0.05,"vader Sentiment Label"] = "negative"
dru_train.shape
dru_train.head()

,ID,review,rating,cleanReview,ReviewScore,Review Score,vader Sentiment,vader Sentiment Label
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000,0.0000,0,neutral
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070,0.9070,2,positive
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096,0.7096,2,positive
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184,0.7184,2,positive
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403,0.9403,2,positive


In [31]:
positive_rating = len(dru_train[dru_train['rating'] >=7.0])
neutral_rating = len(dru_train[(dru_train['rating'] >=4) & (dru_train['rating']<7)])
negative_rating = len(dru_train[dru_train['rating']<=3])
positive_rating,neutral_rating,negative_rating

Total_rating = positive_rating+neutral_rating+negative_rating
Total_rating


215063

In [32]:
dru_train['rating Sentiment']= dru_train['rating'].map(lambda x:int(2) if x>=7 else int(1) if x<=3 else int(0))
dru_train['rating Sentiment'].value_counts()

dru_train.head()

,ID,review,rating,cleanReview,ReviewScore,Review Score,vader Sentiment,vader Sentiment Label,rating Sentiment
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000,0.0000,0,neutral,2
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070,0.9070,2,positive,2
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096,0.7096,2,positive,0
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184,0.7184,2,positive,2
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403,0.9403,2,positive,2


In [34]:
dru_train.loc[dru_train['rating'] >=7.0,"rating Sentiment Label"] ="positive"
dru_train.loc[(dru_train['rating'] >=4.0) & (dru_train['rating']<7.0),"rating Sentiment Label"]= "neutral"
dru_train.loc[dru_train['rating']<=3.0,"rating Sentiment Label"] = "negative"

dru_train.head()

,ID,review,rating,cleanReview,ReviewScore,Review Score,vader Sentiment,vader Sentiment Label,rating Sentiment,rating Sentiment Label,ratingSentimentLabel
0,206461,"""It has no side effect, I take it in combinati...",9.0,"""It side effect, I take combination Bystolic 5...",0.0000,0.0000,0,neutral,2,positive,NaN
1,95260,"""My son is halfway through his fourth week of ...",8.0,"""My son halfway fourth week Intuniv. We became...",0.9070,0.9070,2,positive,2,positive,NaN
2,92703,"""I used to take another oral contraceptive, wh...",5.0,"""I used take another oral contraceptive, 21 pi...",0.7096,0.7096,2,positive,0,neutral,neutral
3,138000,"""This is my first time using any form of birth...",8.0,"""This first time using form birth control. I&#...",0.7184,0.7184,2,positive,2,positive,NaN
4,35696,"""Suboxone has completely turned my life around...",9.0,"""Suboxone completely turned life around. I fee...",0.9403,0.9403,2,positive,2,positive,NaN


In [35]:
dru_train = dru_train[['ID','review','cleanReview','rating','rating Sentiment','rating Sentiment Label',
                       'Review Score','vader Sentiment','vader Sentiment Label']]
dru_train.head()

,ID,review,cleanReview,rating,rating Sentiment,rating Sentiment Label,Review Score,vader Sentiment,vader Sentiment Label
0,206461,"""It has no side effect, I take it in combinati...","""It side effect, I take combination Bystolic 5...",9.0,2,positive,0.0000,0,neutral
1,95260,"""My son is halfway through his fourth week of ...","""My son halfway fourth week Intuniv. We became...",8.0,2,positive,0.9070,2,positive
2,92703,"""I used to take another oral contraceptive, wh...","""I used take another oral contraceptive, 21 pi...",5.0,0,neutral,0.7096,2,positive
3,138000,"""This is my first time using any form of birth...","""This first time using form birth control. I&#...",8.0,2,positive,0.7184,2,positive
4,35696,"""Suboxone has completely turned my life around...","""Suboxone completely turned life around. I fee...",9.0,2,positive,0.9403,2,positive


In [36]:
dru_train.to_csv('newDrug.csv')    # To save preprocessed dataset to csv
dru_train.head(50)
import os
os.stat('newDrug.csv').st_size         # Check size of csv file About 181MB


181826802

In [37]:
dru_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215063 entries, 0 to 53765
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      215063 non-null  int64  
 1   review                  215063 non-null  object 
 2   cleanReview             215063 non-null  object 
 3   rating                  215063 non-null  float64
 4   rating Sentiment        215063 non-null  int64  
 5   rating Sentiment Label  215063 non-null  object 
 6   Review Score            215063 non-null  float64
 7   vader Sentiment         215063 non-null  int64  
 8   vader Sentiment Label   215063 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 16.4+ MB


In [38]:
dru_train.to_csv('newDrug.csv.gz',compression='gzip')
os.stat('newDrug.csv.gz').st_size

54014524

In [39]:
dru_train.head()

,ID,review,cleanReview,rating,rating Sentiment,rating Sentiment Label,Review Score,vader Sentiment,vader Sentiment Label
0,206461,"""It has no side effect, I take it in combinati...","""It side effect, I take combination Bystolic 5...",9.0,2,positive,0.0000,0,neutral
1,95260,"""My son is halfway through his fourth week of ...","""My son halfway fourth week Intuniv. We became...",8.0,2,positive,0.9070,2,positive
2,92703,"""I used to take another oral contraceptive, wh...","""I used take another oral contraceptive, 21 pi...",5.0,0,neutral,0.7096,2,positive
3,138000,"""This is my first time using any form of birth...","""This first time using form birth control. I&#...",8.0,2,positive,0.7184,2,positive
4,35696,"""Suboxone has completely turned my life around...","""Suboxone completely turned life around. I fee...",9.0,2,positive,0.9403,2,positive
